In [ ]:
import numpy as np

from scipy.signal import welch, get_window
from obspy.signal.filter import bandpass

from obspy.clients.filesystem.sds import Client
from obspy.clients.fdsn import RoutingClient
from obspy.core import UTCDateTime as UTC
from obspy.signal import util

import h5py

In [ ]:
import matplotlib.pyplot as plt
plt.style.use('tableau-colorblind10')
#%matplotlib widget

In [ ]:
def get_windows(tr, starttime, nwin, winlen_samples):
    ntot = int(nwin*winlen_samples)
    data = tr.slice(starttime, endtime=None).data[:ntot]
    return data.reshape((nwin, winlen_samples))

In [ ]:
network = 'GR'
station = 'BFO'
location = ''
channel = 'BHZ'
startdate = UTC("2020-107")
#enddate = UTC("2020-109")
overlap = 600 #3600

fmin, fmax = (4, 14)
nperseg = 2048
winlen_in_s = 3600
proclen = 24*3600

dataclient = Client('/home/lehr/sds/data')
invclient = RoutingClient('eida-routing')

In [ ]:
starttime = startdate-overlap
endtime = starttime + proclen + 2*overlap
inv = invclient.get_stations(network=network, station=station, 
                             location=location, starttime=startdate, 
                             endtime=endtime, channel=channel, 
                             level='response')

st = dataclient.get_waveforms(network, station, 
                             location, channel,starttime, endtime)
st.remove_sensitivity(inv)
st.merge(fill_value=np.nan)
tr = st[0]    

# Demean ignoring gaps
tr.data = tr.data - np.nanmean(tr.data)

# Apply filter once

In [ ]:
trfilt = tr.copy().filter('bandpass', freqmin=fmin, freqmax=fmax)

In [ ]:
%%timeit -n100
trfilt = tr.copy().filter('bandpass', freqmin=fmin, freqmax=fmax)

# Filtering blockwise

In [ ]:
sr = tr.stats.sampling_rate
nwin = int(proclen/winlen_in_s)
#proclen_samples = proclen * sr
winlen_samples = int(winlen_in_s * sr)
data = get_windows(tr, starttime+overlap, nwin, winlen_samples)

In [ ]:
%%timeit -n100

matrix = data.copy()
for i, r in enumerate(matrix):
    matrix[i,:] = bandpass(r, fmin, fmax, sr)
    

In [ ]:
while starttime < enddate - overlap:
    print(starttime)
    endtime = starttime + proclen + 2*overlap
    st = dataclient.get_waveforms(network, station, 
                             location, channel,starttime, endtime)
    st.remove_sensitivity(inv)
    st.merge(fill_value=np.nan)
    if len(st) > 1:
        raise RuntimeWarning("More than 1 trace in stream!")
    tr = st[0]    

    # Demean ignoring gaps
    tr.data = tr.data - np.nanmean(tr.data)
    
    break
    # Get some numbers
    sr = tr.stats.sampling_rate
    nwin = int(proclen/winlen_in_s)
    #proclen_samples = proclen * sr
    winlen_samples = int(winlen_in_s * sr)

    # Spectra
    data = get_windows(tr, starttime+overlap, nwin, winlen_samples)
    freq, P = welch(data, fs=sr, nperseg=nperseg, axis=1)

    # Amplitude
    tr.filter('bandpass', freqmin=fmin, freqmax=fmax)
    data = get_windows(tr, starttime+overlap, nwin, winlen_samples)
    prctl = np.nanpercentile(data, 75, axis=1)

    #AMP.append(prctl) #amp[1:-1])
    #PXX.append(P) # pxx[1:-1,:])

    starttime = starttime + proclen